# Введение в MapReduce модель на Python


In [1]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

In [2]:
def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)

def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

Модель элемента данных

In [3]:
class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

In [4]:
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

Функция RECORDREADER моделирует чтение элементов с диска или по сети.

In [5]:
def RECORDREADER():
  return [(u.id, u) for u in input_collection]

In [6]:
list(RECORDREADER())

[(0, User(id=0, age=55, social_contacts=20, gender='male')),
 (1, User(id=1, age=25, social_contacts=240, gender='female')),
 (2, User(id=2, age=25, social_contacts=500, gender='female')),
 (3, User(id=3, age=33, social_contacts=800, gender='female'))]

In [7]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

In [8]:
map_output = flatten(map(lambda x: MAP(*x), RECORDREADER()))
map_output = list(map_output) # materialize
map_output

[(25, User(id=1, age=25, social_contacts=240, gender='female')),
 (25, User(id=2, age=25, social_contacts=500, gender='female')),
 (33, User(id=3, age=33, social_contacts=800, gender='female'))]

In [9]:
def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

In [10]:
shuffle_output = groupbykey(map_output)
shuffle_output = list(shuffle_output)
shuffle_output

[(25,
  [User(id=1, age=25, social_contacts=240, gender='female'),
   User(id=2, age=25, social_contacts=500, gender='female')]),
 (33, [User(id=3, age=33, social_contacts=800, gender='female')])]

In [11]:
reduce_output = flatten(map(lambda x: REDUCE(*x), shuffle_output))
reduce_output = list(reduce_output)
reduce_output

[(25, 370.0), (33, 800.0)]

Все действия одним конвейером!

In [12]:
list(flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER()))))))

[(25, 370.0), (33, 800.0)]

# **MapReduce**
Выделим общую для всех пользователей часть системы в отдельную функцию высшего порядка. Это наиболее простая модель MapReduce, без учёта распределённого хранения данных.

Пользователь для решения своей задачи реализует RECORDREADER, MAP, REDUCE.

In [13]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

## Спецификация MapReduce



```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

mapreduce ((k1,v1)*) -> (k3,v3)*
groupby ((k2,v2)*) -> (k2,v2*)*
flatten (e2**) -> e2*

mapreduce .map(f).flatten.groupby(k2).map(g).flatten
```




# Примеры

## SQL

In [14]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)

def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

def RECORDREADER():
  return [(u.id, u) for u in input_collection]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(25, 370.0), (33, 800.0)]

## Matrix-Vector multiplication

In [15]:
from typing import Iterator
import numpy as np

mat = np.ones((5,4))
vec = np.random.rand(4) # in-memory vector in all map tasks

def MAP(coordinates:(int, int), value:int):
  i, j = coordinates
  yield (i, value*vec[j])

def REDUCE(i:int, products:Iterator[NamedTuple]):
  sum = 0
  for p in products:
    sum += p
  yield (i, sum)

def RECORDREADER():
  for i in range(mat.shape[0]):
    for j in range(mat.shape[1]):
      yield ((i, j), mat[i,j])

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(0, np.float64(1.8796805951418665)),
 (1, np.float64(1.8796805951418665)),
 (2, np.float64(1.8796805951418665)),
 (3, np.float64(1.8796805951418665)),
 (4, np.float64(1.8796805951418665))]

## Inverted index

In [16]:
from typing import Iterator

d1 = "it is what it is"
d2 = "what is it"
d3 = "it is a banana"
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    yield ("{}".format(docid), document)

def MAP(docId:str, body:str):
  for word in set(body.split(' ')):
    yield (word, docId)

def REDUCE(word:str, docIds:Iterator[str]):
  yield (word, sorted(docIds))

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('what', ['0', '1']),
 ('it', ['0', '1', '2']),
 ('is', ['0', '1', '2']),
 ('a', ['2']),
 ('banana', ['2'])]

## WordCount

In [17]:
from typing import Iterator

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    for (lineid, line) in enumerate(document.split('\n')):
      yield ("{}:{}".format(docid,lineid), line)

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('', 3), ('it', 9), ('is', 9), ('what', 5), ('a', 1), ('banana', 1)]

# MapReduce Distributed

Добавляется в модель фабрика RECORDREARER-ов --- INPUTFORMAT, функция распределения промежуточных результатов по партициям PARTITIONER, и функция COMBINER для частичной аггрегации промежуточных результатов до распределения по новым партициям.

In [18]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

## Спецификация MapReduce Distributed


```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

e1 (k1, v1)
e2 (k2, v2)
partition1 (k2, v2)*
partition2 (k2, v2*)*

flatmap (e1->e2*, e1*) -> partition1*
groupby (partition1*) -> partition2*

mapreduce ((k1,v1)*) -> (k3,v3)*
mapreduce .flatmap(f).groupby(k2).flatmap(g)
```



## WordCount

In [19]:
from typing import Iterator
import numpy as np

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3, d1, d2, d3]

maps = 3
reducers = 2

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for (docid, document) in enumerate(split):
      for (lineid, line) in enumerate(document.split('\n')):
        yield ("{}:{}".format(docid,lineid), line)

  split_size =  int(np.ceil(len(documents)/maps))
  for i in range(0, len(documents), split_size):
    yield RECORDREADER(documents[i:i+split_size])

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

# try to set COMBINER=REDUCER and look at the number of values sent over the network
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=REDUCE)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

16 key-value pairs were sent over a network.


[(0, [('', 6), ('banana', 2), ('is', 18)]),
 (1, [('a', 2), ('it', 18), ('what', 10)])]

## TeraSort

In [20]:
import numpy as np

input_values = np.random.rand(30)
maps = 3
reducers = 2
min_value = 0.0
max_value = 1.0

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for value in split:
        yield (value, None)

  split_size =  int(np.ceil(len(input_values)/maps))
  for i in range(0, len(input_values), split_size):
    yield RECORDREADER(input_values[i:i+split_size])

def MAP(value:int, _):
  yield (value, None)

def PARTITIONER(key):
  global reducers
  global max_value
  global min_value
  bucket_size = (max_value-min_value)/reducers
  bucket_id = 0
  while((key>(bucket_id+1)*bucket_size) and ((bucket_id+1)*bucket_size<max_value)):
    bucket_id += 1
  return bucket_id

def REDUCE(value:int, _):
  yield (None,value)

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None, PARTITIONER=PARTITIONER)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

30 key-value pairs were sent over a network.


[(0,
  [(None, np.float64(0.006471586233904092)),
   (None, np.float64(0.015202298253704227)),
   (None, np.float64(0.03524638003872993)),
   (None, np.float64(0.10717362123019258)),
   (None, np.float64(0.13368776885828793)),
   (None, np.float64(0.13581339094477496)),
   (None, np.float64(0.14959090385465035)),
   (None, np.float64(0.18765321376080923)),
   (None, np.float64(0.2564055886463211)),
   (None, np.float64(0.2845867684672234)),
   (None, np.float64(0.30630279704157926)),
   (None, np.float64(0.307792451026344)),
   (None, np.float64(0.33489094938564024)),
   (None, np.float64(0.3942712920437974)),
   (None, np.float64(0.40341604016653)),
   (None, np.float64(0.4121539510857706)),
   (None, np.float64(0.4420833711486367))]),
 (1,
  [(None, np.float64(0.5052484333330882)),
   (None, np.float64(0.5170885485848169)),
   (None, np.float64(0.5355547413322239)),
   (None, np.float64(0.5414944872261009)),
   (None, np.float64(0.5947530071619038)),
   (None, np.float64(0.6030726126

# Упражнения
Упражнения взяты из Rajaraman A., Ullman J. D. Mining of massive datasets. – Cambridge University Press, 2011.


Для выполнения заданий переопределите функции RECORDREADER, MAP, REDUCE. Для модели распределённой системы может потребоваться переопределение функций PARTITION и COMBINER.

### Максимальное значение ряда

Разработайте MapReduce алгоритм, который находит максимальное число входного списка чисел.

In [21]:
import numpy as np

input_values = [np.random.randint(10) for _ in range(10)]
print(f"Input values: {input_values}")

def RECORDREADER():
  for value in input_values:
    yield (None, value)

def MAP(_, value:int):
  yield (None, value)

def REDUCE(_, values:Iterator[int]):
  yield max(values)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
print(f"Max value: {output}")

Input values: [4, 4, 8, 7, 5, 2, 1, 0, 0, 8]
Max value: [8]


### Арифметическое среднее

Разработайте MapReduce алгоритм, который находит арифметическое среднее.

$$\overline{X} = \frac{1}{n}\sum_{i=0}^{n} x_i$$


In [22]:
import numpy as np

input_values = [np.random.randint(10) for _ in range(10)]
print(f"Input values: {input_values}")

def RECORDREADER():
  for value in input_values:
    yield (None, value)

def MAP(_, value:int):
  yield (None, value)

def REDUCE(_, values:Iterator[int]):
  sum, count = 0, 0
  for value in values:
    sum += value
    count += 1
  yield sum/count

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
print(f"Mean: {output}")

Input values: [6, 0, 2, 2, 8, 2, 3, 3, 5, 6]
Mean: [3.7]


### GroupByKey на основе сортировки

Реализуйте groupByKey на основе сортировки, проверьте его работу на примерах

In [23]:
class Student(NamedTuple):
  id: int
  group: str
  name: str

input_collection = [
    Student(id=0, group='B', name='Ivan'),
    Student(id=1, group='A', name='Sasha'),
    Student(id=2, group='B', name='Denis'),
]

def RECORDREADER():
  return [(s.group, s.id) for s in input_collection]

def MAP(group:str, id:int):
  yield(group, id)

def sorted_groupbykey(iterable):
  result = []
  current_key = None
  current_group = []

  for k, v in sorted(iterable, key=lambda x: x[0]):
    if k != current_key:
      if current_key is not None:
        result += [(current_key, current_group)]
      current_key = k
      current_group = [v]
    else:
      current_group += [v]

  if current_key is not None:
    result += [(current_key, current_group)]

  return result

def REDUCE(group:str, stundents:Iterator[NamedTuple]):
  count = 0
  for s in stundents:
    count += 1
  yield(group, count)

map_output = flatten(map(lambda x: MAP(*x), RECORDREADER()))
map_output = list(map_output)
print(f'Map output: {map_output}')

shuffle_output = sorted_groupbykey(map_output)
shuffle_output = list(shuffle_output)
print(f'Shuffle output: {shuffle_output}')

reduce_output = flatten(map(lambda x: REDUCE(*x), shuffle_output))
reduce_output = list(reduce_output)
print(f'Reduce output: {reduce_output}')

Map output: [('B', 0), ('A', 1), ('B', 2)]
Shuffle output: [('A', [1]), ('B', [0, 2])]
Reduce output: [('A', 1), ('B', 2)]


### Drop duplicates (set construction, unique elements, distinct)

Реализуйте распределённую операцию исключения дубликатов

In [24]:
from typing import Iterator
import numpy as np

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3, d1, d2, d3]

maps = 3
reducers = 2

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for (docid, document) in enumerate(split):
      for (lineid, line) in enumerate(document.split('\n')):
        yield ("{}:{}".format(docid,lineid), line)

  split_size =  int(np.ceil(len(documents)/maps))
  for i in range(0, len(documents), split_size):
    yield RECORDREADER(documents[i:i+split_size])

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word:str, counts:Iterator[int]):
  yield word

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

56 key-value pairs were sent over a network.


[(0, ['', 'banana', 'is']), (1, ['a', 'it', 'what'])]

#Операторы реляционной алгебры
### Selection (Выборка)

**The Map Function**: Для  каждого кортежа $t \in R$ вычисляется истинность предиката $C$. В случае истины создаётся пара ключ-значение $(t, t)$. В паре ключ и значение одинаковы, равны $t$.

**The Reduce Function:** Роль функции Reduce выполняет функция идентичности, которая возвращает то же значение, что получила на вход.



In [25]:
import numpy as np

R = [(np.random.randint(10), np.random.randint(10))  for _ in range(10)]
print(f"Input values: {R}")

def C(t):
  return sum(t) > 10

def RECORDREADER():
  for t in R:
    yield (None, t)

def MAP(_, t):
  if C(t):
    yield (t, t)

def REDUCE(k, v):
  yield (k, v)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
print(f"Output: {output}")

Input values: [(9, 8), (1, 3), (1, 2), (9, 7), (5, 7), (9, 7), (2, 0), (5, 4), (2, 3), (5, 1)]
Output: [((9, 8), [(9, 8)]), ((9, 7), [(9, 7), (9, 7)]), ((5, 7), [(5, 7)])]


### Projection (Проекция)

Проекция на множество атрибутов $S$.

**The Map Function:** Для каждого кортежа $t \in R$ создайте кортеж $t′$, исключая  из $t$ те значения, атрибуты которых не принадлежат  $S$. Верните пару $(t′, t′)$.

**The Reduce Function:** Для каждого ключа $t′$, созданного любой Map задачей, вы получаете одну или несколько пар $(t′, t′)$. Reduce функция преобразует $(t′, [t′, t′, . . . , t′])$ в $(t′, t′)$, так, что для ключа $t′$ возвращается одна пара  $(t′, t′)$.

In [26]:
import numpy as np

R = [(np.random.randint(10), np.random.randint(10))  for _ in range(10)]
print(f"Input values: {R}")

S = [1, 2, 3, 4]

def RECORDREADER():
  for t in R:
    yield (None, t)

def MAP(_, t):
  values = []
  for value in t:
    if value in S:
      values += [value]

  t_new = tuple(values)
  yield (t_new, t_new)

def REDUCE(k, v):
  yield (k, k)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
print(f"Result: {output}")

Input values: [(3, 2), (3, 8), (2, 5), (9, 1), (4, 1), (2, 7), (0, 8), (5, 5), (8, 7), (9, 7)]
Result: [((3, 2), (3, 2)), ((3,), (3,)), ((2,), (2,)), ((1,), (1,)), ((4, 1), (4, 1)), ((), ())]


### Union (Объединение)

**The Map Function:** Превратите каждый входной кортеж $t$ в пару ключ-значение $(t, t)$.

**The Reduce Function:** С каждым ключом $t$ будет ассоциировано одно или два значений. В обоих случаях создайте $(t, t)$ в качестве выходного значения.

In [27]:
import numpy as np

R = [(np.random.randint(10), np.random.randint(10))  for _ in range(10)]
print(f"Input values: {R}")

def RECORDREADER():
  for t in R:
    yield (None, t)

def MAP(_, t):
  yield (t, t)

def REDUCE(k, v):
  yield (k, k)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
print(f"Result: {output}")

Input values: [(4, 2), (2, 4), (5, 2), (4, 0), (6, 4), (5, 7), (5, 9), (3, 6), (7, 1), (2, 2)]
Result: [((4, 2), (4, 2)), ((2, 4), (2, 4)), ((5, 2), (5, 2)), ((4, 0), (4, 0)), ((6, 4), (6, 4)), ((5, 7), (5, 7)), ((5, 9), (5, 9)), ((3, 6), (3, 6)), ((7, 1), (7, 1)), ((2, 2), (2, 2))]


### Intersection (Пересечение)

**The Map Function:** Превратите каждый кортеж $t$ в пары ключ-значение $(t, t)$.

**The Reduce Function:** Если для ключа $t$ есть список из двух элементов $[t, t]$ $-$ создайте пару $(t, t)$. Иначе, ничего не создавайте.

In [28]:
import numpy as np

R = [(np.random.randint(2), np.random.randint(2))  for _ in range(10)]
print(f"Input values: {R}")

def RECORDREADER():
  for t in R:
    yield (None, t)

def MAP(_, t):
  yield (t, t)

def REDUCE(k, v):
  if len(v) == 2:
    yield (k, k)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
print(f"Result: {output}")

Input values: [(0, 1), (1, 1), (1, 0), (0, 0), (0, 1), (0, 1), (0, 0), (0, 1), (1, 0), (1, 1)]
Result: [((1, 1), (1, 1)), ((1, 0), (1, 0)), ((0, 0), (0, 0))]


### Difference (Разница)

**The Map Function:** Для кортежа $t \in R$, создайте пару $(t, R)$, и для кортежа $t \in S$, создайте пару $(t, S)$. Задумка заключается в том, чтобы значение пары было именем отношения $R$ or $S$, которому принадлежит кортеж (а лучше, единичный бит, по которому можно два отношения различить $R$ or $S$), а не весь набор атрибутов отношения.

**The Reduce Function:** Для каждого ключа $t$, если соответствующее значение является списком $[R]$, создайте пару $(t, t)$. В иных случаях не предпринимайте действий.

In [29]:
import numpy as np

input_values = [(np.random.randint(10), np.random.randint(10), np.random.randint(2))  for _ in range(10)]
print(f"Input values: {input_values}")

def RECORDREADER():
  for t in input_values:
    yield ((t[0], t[1]), t[2])

def MAP(t, id):
  yield (t, id)

def REDUCE(t, id):
  if id == [0]:
    yield (t, t)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
print(f"Result: {output}")

Input values: [(3, 1, 0), (1, 1, 0), (8, 0, 0), (8, 5, 0), (3, 3, 0), (7, 3, 0), (7, 4, 0), (5, 1, 0), (0, 9, 0), (1, 0, 1)]
Result: [((3, 1), (3, 1)), ((1, 1), (1, 1)), ((8, 0), (8, 0)), ((8, 5), (8, 5)), ((3, 3), (3, 3)), ((7, 3), (7, 3)), ((7, 4), (7, 4)), ((5, 1), (5, 1)), ((0, 9), (0, 9))]


### Natural Join

**The Map Function:** Для каждого кортежа $(a, b)$ отношения $R$, создайте пару $(b,(R, a))$. Для каждого кортежа $(b, c)$ отношения $S$, создайте пару $(b,(S, c))$.

**The Reduce Function:** Каждый ключ $b$ будет асоциирован со списком пар, которые принимают форму либо $(R, a)$, либо $(S, c)$. Создайте все пары, одни, состоящие из  первого компонента $R$, а другие, из первого компонента $S$, то есть $(R, a)$ и $(S, c)$. На выходе вы получаете последовательность пар ключ-значение из списков ключей и значений. Ключ не нужен. Каждое значение, это тройка $(a, b, c)$ такая, что $(R, a)$ и $(S, c)$ это принадлежат входному списку значений.

In [30]:
import numpy as np

input_values = [(np.random.randint(10), np.random.randint(10), np.random.randint(2))  for _ in range(10)]
print(f"Input values: {input_values}")

def RECORDREADER():
  for t in input_values:
    yield ((t[0], t[1]), t[2])

def MAP(t, id):
  if id == [0]:
    yield ((id, t[0]), t[1])
  else:
    yield ((id, t[1]), t[0])

def REDUCE(t:Iterator[int], id:int):
  res = []
  for v in t:
    res += [v]
  yield tuple(res)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
print(f"Result: {output}")

Input values: [(5, 9, 1), (2, 9, 1), (2, 8, 1), (3, 6, 1), (4, 1, 0), (7, 2, 1), (5, 3, 1), (3, 4, 1), (3, 3, 1), (1, 3, 0)]
Result: [(1, 9), (1, 8), (1, 6), (0, 1), (1, 2), (1, 3), (1, 4), (0, 3)]


### Grouping and Aggregation (Группировка и аггрегация)

**The Map Function:** Для каждого кортежа $(a, b, c$) создайте пару $(a, b)$.

**The Reduce Function:** Ключ представляет ту или иную группу. Примение аггрегирующую операцию $\theta$ к списку значений $[b1, b2, . . . , bn]$ ассоциированных с ключом $a$. Возвращайте в выходной поток $(a, x)$, где $x$ результат применения  $\theta$ к списку. Например, если $\theta$ это $SUM$, тогда $x = b1 + b2 + · · · + bn$, а если $\theta$ is $MAX$, тогда $x$ это максимальное из значений $b1, b2, . . . , bn$.

In [31]:
import numpy as np

input_values = [(np.random.randint(10), np.random.randint(10), np.random.randint(10))  for _ in range(10)]
print(f"Input values: {input_values}")

def RECORDREADER():
  for t in input_values:
    yield (None, (t[0], t[1], t[2]))

def MAP(_, t):
  yield (t[0], t[1])

def REDUCE(k, v:Iterator[int]):
    yield (k, sum(v))

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
print(f"Result: {output}")

Input values: [(4, 5, 6), (1, 1, 9), (3, 8, 6), (4, 6, 7), (9, 2, 4), (3, 8, 6), (4, 7, 5), (0, 2, 4), (6, 0, 3), (5, 2, 7)]
Result: [(4, 18), (1, 1), (3, 16), (9, 2), (0, 2), (6, 0), (5, 2)]


#

### Matrix-Vector multiplication

Случай, когда вектор не помещается в памяти Map задачи


## Matrix multiplication (Перемножение матриц)

Если у нас есть матрица $M$ с элементами $m_{ij}$ в строке $i$ и столбце $j$, и матрица $N$ с элементами $n_{jk}$ в строке $j$ и столбце $k$, тогда их произведение $P = MN$ есть матрица $P$ с элементами $p_{ik}$ в строке $i$ и столбце $k$, где

$$p_{ik} =\sum_{j} m_{ij}n_{jk}$$

Необходимым требованием является одинаковое количество столбцов в $M$ и строк в $N$, чтобы операция суммирования по  $j$ была осмысленной. Мы можем размышлять о матрице, как об отношении с тремя атрибутами: номер строки, номер столбца, само значение. Таким образом матрица $M$ предстваляется как отношение $ M(I, J, V )$, с кортежами $(i, j, m_{ij})$, и, аналогично, матрица $N$ представляется как отношение $N(J, K, W)$, с кортежами $(j, k, n_{jk})$. Так как большие матрицы как правило разреженные (большинство значений равно 0), и так как мы можем нулевыми значениями пренебречь (не хранить), такое реляционное представление достаточно эффективно для больших матриц. Однако, возможно, что координаты $i$, $j$, и $k$ неявно закодированы в смещение позиции элемента относительно начала файла, вместо явного хранения. Тогда, функция Map (или Reader) должна быть разработана таким образом, чтобы реконструировать компоненты $I$, $J$, и $K$ кортежей из смещения.

Произведение $MN$ это фактически join, за которым следуют группировка по ключу и аггрегация. Таким образом join отношений $M(I, J, V )$ и $N(J, K, W)$, имеющих общим только атрибут $J$, создаст кортежи $(i, j, k, v, w)$ из каждого кортежа $(i, j, v) \in M$ и кортежа $(j, k, w) \in N$. Такой 5 компонентный кортеж представляет пару элементов матрицы $(m_{ij} , n_{jk})$. Что нам хотелось бы получить на самом деле, это произведение этих элементов, то есть, 4 компонентный кортеж$(i, j, k, v \times w)$, так как он представляет произведение $m_{ij}n_{jk}$. Мы представляем отношение как результат одной MapReduce операции, в которой мы можем произвести группировку и аггрегацию, с $I$ и $K$  атрибутами, по которым идёт группировка, и суммой  $V \times W$.





In [32]:
# MapReduce model
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

Реализуйте перемножение матриц с использованием модельного кода MapReduce для одной машины в случае, когда одна матрица хранится в памяти, а другая генерируется RECORDREADER-ом.

In [33]:
import numpy as np
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J) # it is legal to access this from RECORDREADER, MAP, REDUCE
big_mat = np.random.rand(J,K)

def RECORDREADER():
  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      yield ((j,k), big_mat[j,k])

def MAP(k1, v1):
  (j, k) = k1
  w = v1
  for i in range(small_mat.shape[0]):
    yield ((i, k), w * small_mat[i][j])

def REDUCE(key, values):
  (i, k) = key
  el_value = 0
  for v in values:
    el_value += v
  yield ((i, k), el_value)

Проверьте своё решение

In [34]:
# CHECK THE SOLUTION
reference_solution = np.matmul(small_mat, big_mat)
solution = MapReduce(RECORDREADER, MAP, REDUCE)

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output)+1
  K = max(k for ((i,k), vw) in reduce_output)+1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

np.allclose(reference_solution, asmatrix(solution))# should return true

True

In [35]:
reduce_output = list(MapReduce(RECORDREADER, MAP, REDUCE))
max(i for ((i,k), vw) in reduce_output)

1

Реализуйте перемножение матриц  с использованием модельного кода MapReduce для одной машины в случае, когда обе матрицы генерируются в RECORDREADER. Например, сначала одна, а потом другая.

In [36]:
import numpy as np

I = 2
J = 3
K = 4 * 10

small_mat = np.random.rand(I, J)
big_mat = np.random.rand(J, K)
reference_solution = np.matmul(small_mat, big_mat)

def RECORDREADER():
  for i in range(small_mat.shape[0]):
    for j in range(small_mat.shape[1]):
      yield ((0, i, j), small_mat[i, j])

  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      yield ((1, j, k), big_mat[j, k])

def MAP_JOIN(k1, v1):
  (mat_num, i, j) = k1
  if mat_num == 0:
    yield (j, (mat_num, i, v1))
  else:
    yield (i, (mat_num, j, v1))

def REDUCE_JOIN(key, values):
  from_first_mat = [v for v in values if v[0] == 0]
  from_second_mat = [v for v in values if v[0] == 1]
  for f in from_first_mat:
    for s in from_second_mat:
      yield ((f[1], s[1]), f[2] * s[2])

def MAP_MUL(k1, v1):
  (i, k) = k1
  yield ((i, k), v1)

def REDUCE_MUL(key, values):
  res_el_value = sum(values)
  yield (key, res_el_value)

def GET_JOINED():
  for j in joined:
    yield j

joined = MapReduce(RECORDREADER, MAP_JOIN, REDUCE_JOIN)
solution = MapReduce(GET_JOINED, MAP_MUL, REDUCE_MUL)
np.allclose(reference_solution, asmatrix(solution))

True

Реализуйте перемножение матриц с использованием модельного кода MapReduce Distributed, когда каждая матрица генерируется в своём RECORDREADER.

In [37]:
import numpy as np

I = 2
J = 3
K = 4 * 10

small_mat = np.random.rand(I, J)
big_mat = np.random.rand(J, K)
reference_solution = np.matmul(small_mat, big_mat)

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output) + 1
  K = max(k for ((i,k), vw) in reduce_output) + 1
  mat = np.empty(shape=(I, K))
  for ((i,k), vw) in reduce_output:
    mat[i, k] = vw
  return mat

def INPUTFORMAT():
  first_mat = []
  for i in range(small_mat.shape[0]):
    for j in range(small_mat.shape[1]):
      first_mat.append(((0, i, j), small_mat[i,j]))
  yield first_mat

  second_mat = []
  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      second_mat.append(((1, j, k), big_mat[j,k]))
  yield second_mat

def MAP_JOIN(k1, v1):
  (mat_num, i, j) = k1
  w = v1
  if mat_num == 0:
    yield (j, (mat_num, i, w))
  else:
    yield (i, (mat_num, j, w))

def REDUCE_JOIN(key, values):
  from_first_mat = [v for v in values if v[0] == 0]
  from_second_mat = [v for v in values if v[0] == 1]
  for f in from_first_mat:
    for s in from_second_mat:
      yield ((f[1], s[1]), f[2] * s[2])

def GET_JOINED():
  for j in joined:
    yield j[1]

def MAP_MUL(k1, v1):
  yield (k1, v1)

def REDUCE_MUL(key, values):
  res_val = 0
  for v in values:
    res_val += v
  yield (key, res_val)

maps = 4
reducers = 2

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP_JOIN, REDUCE_JOIN, COMBINER=None)
joined = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
print(joined)

mul_output = MapReduceDistributed(GET_JOINED, MAP_MUL, REDUCE_MUL, COMBINER=None)
pre_result = [(partition_id, list(partition)) for (partition_id, partition) in mul_output]
print(pre_result)

solution = []
for p in pre_result:
  for v in p[1]:
    solution.append(v)

print(solution)

np.allclose(reference_solution, asmatrix(solution))

126 key-value pairs were sent over a network.
[(0, [((0, 0), np.float64(0.05211737733686963)), ((0, 1), np.float64(0.10559579641544001)), ((0, 2), np.float64(0.03428671585794558)), ((0, 3), np.float64(0.08482528303143635)), ((0, 4), np.float64(0.12105060011032226)), ((0, 5), np.float64(0.057983776945162456)), ((0, 6), np.float64(0.0018205144563275923)), ((0, 7), np.float64(0.07503209573772306)), ((0, 8), np.float64(0.09275020679938337)), ((0, 9), np.float64(0.11781690982034672)), ((0, 10), np.float64(0.10312410621621473)), ((0, 11), np.float64(0.0824881451896362)), ((0, 12), np.float64(0.00896347173295578)), ((0, 13), np.float64(0.04911253601344081)), ((0, 14), np.float64(0.03594843476228986)), ((0, 15), np.float64(0.09905937976253398)), ((0, 16), np.float64(0.1171145773904806)), ((0, 17), np.float64(0.09114861611339006)), ((0, 18), np.float64(0.1136296214251542)), ((0, 19), np.float64(0.11980433565417467)), ((0, 20), np.float64(0.007666601843827331)), ((0, 21), np.float64(0.0867295200

True

Обобщите предыдущее решение на случай, когда каждая матрица генерируется несколькими RECORDREADER-ами, и проверьте его работоспособность. Будет ли работать решение, если RECORDREADER-ы будут генерировать случайное подмножество элементов матрицы?

In [38]:
import numpy as np

I = 2
J = 3
K = 4 * 10

small_mat = np.random.rand(I, J)
big_mat = np.random.rand(J, K)
reference_solution = np.matmul(small_mat, big_mat)

def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER)
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)
  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output) + 1
  K = max(k for ((i,k), vw) in reduce_output) + 1
  mat = np.empty(shape=(I, K))
  for ((i,k), vw) in reduce_output:
    mat[i, k] = vw
  return mat

def INPUTFORMAT():
  first_mat = []
  for i in range(small_mat.shape[0]):
    for j in range(small_mat.shape[1]):
      first_mat.append(((0, i, j), small_mat[i,j]))
  global maps
  split_size = int(np.ceil(len(first_mat)/maps))
  for i in range(0, len(first_mat), split_size):
    yield first_mat[i:i+split_size]
  second_mat = []
  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      second_mat.append(((1, j, k), big_mat[j,k]))
  split_size = int(np.ceil(len(second_mat)/maps))
  for i in range(0, len(second_mat), split_size):
    yield second_mat[i:i+split_size]

def MAP_JOIN(k1, v1):
  (mat_num, i, j) = k1
  w = v1
  if mat_num == 0:
    yield (j, (mat_num, i, w))
  else:
    yield (i, (mat_num, j, w))

def REDUCE_JOIN(key, values):
  from_first_mat = [v for v in values if v[0] == 0]
  from_second_mat = [v for v in values if v[0] == 1]
  for f in from_first_mat:
    for s in from_second_mat:
      yield ((f[1], s[1]), f[2] * s[2])

def GET_JOINED():
  for j in joined:
    yield j[1]

def MAP_MUL(k1, v1):
  yield (k1, v1)

def REDUCE_MUL(key, values):
  res_val = 0
  for v in values:
    res_val += v
  yield (key, res_val)

maps = 3
reducers = 2

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP_JOIN, REDUCE_JOIN, COMBINER=None)
joined = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
print(joined)

mul_output = MapReduceDistributed(GET_JOINED, MAP_MUL, REDUCE_MUL, COMBINER=None)
pre_result = [(partition_id, list(partition)) for (partition_id, partition) in mul_output]
print(pre_result)

solution = []
for p in pre_result:
  for v in p[1]:
    solution.append(v)

print(solution)

np.allclose(reference_solution, asmatrix(solution))

126 key-value pairs were sent over a network.
[(0, [((0, 0), np.float64(0.3130948031353835)), ((0, 1), np.float64(0.5857384131560364)), ((0, 2), np.float64(0.3834015719096146)), ((0, 3), np.float64(0.17630443461069834)), ((0, 4), np.float64(0.40402716652324827)), ((0, 5), np.float64(0.46092245790160624)), ((0, 6), np.float64(0.0995040113513068)), ((0, 7), np.float64(0.6755809431241886)), ((0, 8), np.float64(0.6446517705767176)), ((0, 9), np.float64(0.629697264337617)), ((0, 10), np.float64(0.10465479043087837)), ((0, 11), np.float64(0.08884076945691988)), ((0, 12), np.float64(0.6190304698752774)), ((0, 13), np.float64(0.037778956935190074)), ((0, 14), np.float64(0.47314966472151504)), ((0, 15), np.float64(0.28732532651876264)), ((0, 16), np.float64(0.20773097679865649)), ((0, 17), np.float64(0.3167099450876009)), ((0, 18), np.float64(0.09691216876735831)), ((0, 19), np.float64(0.049714461331308434)), ((0, 20), np.float64(0.3627368467043575)), ((0, 21), np.float64(0.4718715887441142)), 

True